# Cumulative Dose volume with pie charts
## Purpose
The purpose of this code is to visualise the cumulative dose of one or multiple files in a pie chart instead of the usual histogram plot.  
Displayed is:
1. The percentage of particles getting the threshold Dose or above (**absolute value**)
2. The percentage of particles getting a specific amount of the dose or more (**relative** to the max value)
## Usage
1. Input the amount of different files you'd like to compare in the code below.
   - There will be one chart per file, not layered.
   - The files share one common slider to compare the percentages
2. Execute the first **three** cells.
3. Input the files and choose names for them
    - The files need to be *.npz* files containing certain data.
    - To compute the data, please use the other provided program with *cdvpie.py* as math_op file.
4. Execute the rest of the cells.
5. Use the slider or input a value in the field next to it.

In [1]:
# imports
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import SimpleITK as sitk

Replace the number below with the amount of files you'd like to compare:

In [9]:
files_to_compare = 2

1. Execute the cell below
2. Fill the input fields
3. Press read input

In [10]:
# list for the images
cdvp_list = []

# list for names
names_list = []

# create input fields for the files
hBoxes = []

for i in range (files_to_compare):
    input_cdvp = widgets.Text(placeholder='Select cdv-pie file (.npz)')
    cdvp_list.append(input_cdvp)
    input_name = widgets.Text(placeholder='Select name')
    names_list.append(input_name)

    hbox = widgets.HBox([input_cdvp, input_name])
    hBoxes.append(hbox)

# display boxes
for i in range (files_to_compare):
    display(hBoxes[i])    

def read_input_values():
    cdvp_values = [widget.value for widget in cdvp_list]
    name_values = [widget.value for widget in names_list]
    return cdvp_values, name_values

# button click event handler
def on_button_click(b):
    global cdvp_list, names_list
    cdvp_list, names_list = read_input_values()

read_button = widgets.Button(description='Read Inputs')
read_button.on_click(on_button_click)

# display the button
display(read_button)

Button(description='Read Inputs', style=ButtonStyle())

## **Please specify files and press the button NOW. THEN execute the next field**

In [11]:
# load cdv-pies
cdvp_list = [cdvp.replace('"', '') for cdvp in cdvp_list]
cdvp_data = [np.load(file, allow_pickle=True) for file in cdvp_list]

# extract data from the cdvhs
flattened_images = [data['flat_image'] for data in cdvp_data]
sorted_images = [data['sorted_image'] for data in cdvp_data]
cumulative_sums = [data['cumsum_image'] for data in cdvp_data]

In [12]:
# Calculate the maximum values for each array
max_value_list = [np.max(image) for image in flattened_images]
overall_max_value = max(max_value_list)


## Absolute Threshold
Please use the slider for threshold adjustion.

In [13]:
def update_plot(threshold):
    global sorted_images
    global overall_max_value
    # clear the current plot
    plt.clf()

    # get the amount of images
    num_images = len(sorted_images)
    
    # visualization
    labels = [f'>= {threshold}GBq', f'< {threshold} GBq']
    fig, axes = plt.subplots(1, num_images, figsize=(6 * num_images, 6))
    
    if num_images == 1:
        axes = [axes]
    
    for idx, (image, ax) in enumerate(zip(sorted_images, axes)):
        count_above_threshold = np.sum(image >= threshold)
        total_count = image.size
        percentage_above_threshold = (count_above_threshold / total_count) * 100
        sizes = [percentage_above_threshold, 100 - percentage_above_threshold]
        
        ax.pie(sizes, labels=labels, autopct='%1.6f%%', startangle=90)
        ax.axis('equal')  
        ax.set_title(f'{names_list[idx]}: Voxel percent with Value >= {threshold} Gy')
        # ax.set_title(f'Array {names_list[idx]}: Percentage of Data Points with Value >= {threshold}')

    plt.show()

# create a slider widget
threshold_slider = widgets.FloatSlider(value=30.0, min=0.0, max=overall_max_value, step=0.01, description='Cumulative Dose %:')

# use the interact function to update the plot based on the slider value
interact(update_plot, threshold=threshold_slider)

interactive(children=(FloatSlider(value=30.0, description='Cumulative Dose %:', max=2715.010004208845, step=0.…

<function __main__.update_plot(threshold)>

## Relative Threshold
Please use the slider for threshold adjustion.

In [14]:
def update_cm_plot(threshold_percentage):
    global sorted_images, cumulative_sums, flattened_images, names_list
    # clear the current plot
    plt.clf()

    # get the amount of images
    num_images = len(flattened_images)
    
    # visualization
    fig, axes = plt.subplots(1, num_images, figsize=(6 * num_images, 6))
    
    if num_images == 1:
        axes = [axes]

    for idx, (sorted_image, cumulative, flattened_image, ax) in enumerate(zip(sorted_images, cumulative_sums, flattened_images, axes)):
        cumulative_percentage = cumulative / cumulative[-1] * 100

        threshold_index = np.searchsorted(cumulative_percentage, threshold_percentage)
        threshold_value = sorted_image[threshold_index] if threshold_index < len(sorted_image) else sorted_image[-1]

        count_above_threshold = np.sum(flattened_image >= threshold_value)
        total_count = flattened_image.size
        percentage_above_threshold = (count_above_threshold / total_count) * 100
        sizes = [percentage_above_threshold, 100 - percentage_above_threshold]
        
        labels = [f'>= {threshold_value:.5f} Gy', f'< {threshold_value:.5f} Gy']
        
        ax.pie(sizes, labels=labels, autopct='%1.12f%%', startangle=90)
        ax.axis('equal')  
        ax.set_title(f'{names_list[idx]}: Cumulative Dose >= {threshold_percentage}%')
        
        # add data (change preciseness of floats here! .Xf)
        text_x = 3  # x-coordinate for the text
        text_y = 1.0 - (idx * 0.1)  # y-coordinate for the text

    plt.show();

# create a float slider widget (int is too imprecise if high values are getting interesting)
threshold_slider = widgets.FloatSlider(value=30.0, min=0.0, max=100.0, step=0.01, description='Cumulative Dose %:')

# use the interact function to update the plot based on the slider value
interact(update_cm_plot, threshold_percentage=threshold_slider)


interactive(children=(FloatSlider(value=30.0, description='Cumulative Dose %:', step=0.01), Output()), _dom_cl…

<function __main__.update_cm_plot(threshold_percentage)>